## Нужные в этом тяжёлом деле (и не очень) библиотеки и модули

In [42]:
import numpy as np
import random as rd 
import simpy
import pandas as pd
import scipy as sp
import statsmodels 

## Генерация случайной величины
Экспоненциальное распределение с вводимым параметром lambda

In [99]:
lambda_ = float(input())
print(np.log(rd.random()) / lambda_)
print(rd.expovariate(1.0 / 4))

-0.036357505624914355
9.309268311370472


## Простой процесс на simpy

In [ ]:
def car(env):
    while True:
        print(f'Start parking at {env.now}')
        parking_duration = 5
        yield env.timeout(parking_duration)

        print(f'Start driving at {env.now}')
        trip_duration = 2
        yield env.timeout(trip_duration)

env = simpy.Environment()
env.process(car(env))
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


## Запуск подпроцесса charge после парковки

In [ ]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print(f'{id(self)} Start parking and charging at {self.env.now}')
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.charge(charge_duration))

            # The charge process has finished and
            # we can start driving again.
            print(f'{id(self)} Start driving at {self.env.now}')
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

env = simpy.Environment()
car_1 = Car(env)
car_2 = Car(env)
env.run(until=15)

2391380788896 Start parking and charging at 0
2391380787648 Start parking and charging at 0
2391380788896 Start driving at 5
2391380787648 Start driving at 5
2391380788896 Start parking and charging at 7
2391380787648 Start parking and charging at 7
2391380788896 Start driving at 12
2391380787648 Start driving at 12
2391380788896 Start parking and charging at 14
2391380787648 Start parking and charging at 14


## Прерывание другого процесса

In [ ]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print(f'{id(self)} Start parking and charging at {self.env.now}')
            charge_duration = 5
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print(f'{id(self)} Was interrupted. Hope, the battery is full enough ...')

            # The charge process has finished and
            # we can start driving again.
            print(f'{id(self)} Start driving at {self.env.now}')
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)


def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

env = simpy.Environment()
car_1 = Car(env)
car_2 = Car(env)
env.process(driver(env, car_1))
env.run(until=15)

2390860442352 Start parking and charging at 0
2391144724560 Start parking and charging at 0
2390860442352 Was interrupted. Hope, the battery is full enough ...
2390860442352 Start driving at 3
2390860442352 Start parking and charging at 5
2391144724560 Start driving at 5
2391144724560 Start parking and charging at 7
2390860442352 Start driving at 10
2390860442352 Start parking and charging at 12
2391144724560 Start driving at 12
2391144724560 Start parking and charging at 14


## Использование ресурсов
Добавляется устройство обслуживания с ёмкостью 2.
Машины 1 и 2 начинают обслуживаться сразу же по прибытии, а 3 и 4 ожидают освобождения слотов для обслуживания.

In [44]:
def car(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to the BCS
    yield env.timeout(driving_time)
    # Request one of its charging spots
    print(f'{name} arriving at {env.now}')
    with bcs.request() as req:
        yield req
        # Charge the battery
        print(f'{name} starting to charge at {env.now}')
        yield env.timeout(charge_duration)
        print(f'{name} leaving the bcs at {env.now}')

env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)
for i in range(4):
    env.process(car(env, f'Car {i+1}', bcs, i*2, 5))
env.run()

Car 1 arriving at 0
Car 1 starting to charge at 0
Car 2 arriving at 2
Car 2 starting to charge at 2
Car 3 arriving at 4
Car 1 leaving the bcs at 5
Car 3 starting to charge at 5
Car 4 arriving at 6
Car 2 leaving the bcs at 7
Car 4 starting to charge at 7
Car 3 leaving the bcs at 10
Car 4 leaving the bcs at 12


## Пример с банком
В банке есть сотрудники, обслуживающие клиентов, и терминалы, выдающие талоны. Есть, соответственно два процесса: выдача талона и обслуживание клиента. 

In [50]:
class Bank(object):
    def __init__(self, env, num_employee, num_terminal):
        self.env = env
        self.employee = simpy.Resource(env, num_employee)
        self.terminal = simpy.Resource(env, num_terminal)

    def service(self, customer):
    # Обслуживание занимает время в интервале от 1 до 15 минут
        yield self.env.timeout(rd.randint(1, 15))

    def take_token(self, customer):
    # Получение талона занимает до 45 секунд
        yield self.env.timeout(45/60)

Создадим функцию, которая будет отвечать за поведение клиента в среде:

In [51]:
def go_to_bank(env, customer, bank):
	# Клиент пришел в банк
	arrival_time = env.now
	
	with bank.terminal.request() as request:
		yield request
		yield env.process(bank.take_token(customer))

	with bank.employee.request() as request:
		yield request
		yield env.process(bank.service(customer))

	global times
	times.append(env.now - arrival_time)

Теперь необходимо создать функцию для запуска моделирования. Она будет отвечать за создание экземпляра банка и генерацию клиентов, до тех пор, пока симуляция не остановится:

In [52]:
def run_bank(env, num_employee, num_terminal):
	bank = Bank(env, num_employee, num_terminal)
	customer = 1
	
	env.process(go_to_bank(env, customer, bank))
	
	while True:
		# Предположим каждые 4 минуты заходит новый клиент
		yield env.timeout(rd.expovariate(1.0 / 240))
		customer += 1
		env.process(go_to_bank(env, customer, bank))

Отдельно создадим функцию для подсчета среднего времени:

In [53]:
def get_average_time(times):
	average_time = np.mean(times)
	minutes, frac_minutes = divmod(average_time, 1)
	seconds = frac_minutes * 60
	return round(minutes), round(seconds)

Запустим моделирование:

In [54]:
rd.seed(42)
# Начальные данные
num_employee = 7 # В офисе работает 7 сотрудников
num_terminal = 1 # В офисе 1 терминал по выдаче талонов
times = []

# Запуск моделирования
env = simpy.Environment()
env.process(run_bank(env, num_employee, num_terminal))
env.run(until=72)

# Результаты
mins, secs = get_average_time(times)
print(f'\nСреднее время обслуживания: {mins}:{secs}')


Среднее время обслуживания: 1:45


## СМО M/M/1

In [100]:
class System(object):
    def __init__(self, env, num_services):
        self.env = env
        self.server = simpy.Resource(env, num_services)

    def service(self, demand):
        yield self.env.timeout(rd.expovariate(1.0 / 2))
        print(f'Требование {id(demand)} обслужено в {self.env.now}')

In [101]:
def go_to_system(env, demand, system):
	# требование пришло в систему
	arrival_time = env.now
	
	with system.server.request() as request:
		yield request
		yield env.process(system.service(demand))

	global times
	times.append(env.now - arrival_time)

In [102]:
def run_system(env, num_services):
	system = System(env, num_services)
	demand = 1
	print(f'Требование {id(demand)} поступило в {env.now}')
	env.process(go_to_system(env, demand, system))
	
	while True:
		yield env.timeout(rd.expovariate(1.0 / 1))
		demand += 1
		print(f'Требование {id(demand)} поступило в {env.now}')
		env.process(go_to_system(env, demand, system))

In [103]:
def get_average_time(times):
	average_time = np.mean(times)
	minutes, frac_minutes = divmod(average_time, 1)
	seconds = frac_minutes * 60
	return round(minutes), round(seconds)

In [104]:
# Начальные данные
num_services = 2 
times = []

# Запуск моделирования
env = simpy.Environment()
env.process(run_system(env, num_services))
env.run(until=7)

# Результаты
mins, secs = get_average_time(times)
print(f'\nСреднее время обслуживания: {mins}:{secs}')

Требование 2390807570736 поступило в 0
Требование 2390807570736 обслужено в 0.004315409044931036
Требование 2390807570768 поступило в 0.17988906338818464
Требование 2390807570800 поступило в 0.6748770302159032
Требование 2390807570800 обслужено в 1.3465197682302805
Требование 2390807570832 поступило в 2.212596121647633
Требование 2390807570864 поступило в 3.4052713301528024
Требование 2390807570832 обслужено в 4.835009360642372
Требование 2390807570896 поступило в 4.934796674847141
Требование 2390807570768 обслужено в 5.4013206669238745
Требование 2390807570928 поступило в 5.601635790614894
Требование 2390807570896 обслужено в 5.8225105658663985
Требование 2390807570960 поступило в 5.847153919333314
Требование 2390807570928 обслужено в 5.943036935667164
Требование 2390807570960 обслужено в 6.068826167664431

Среднее время обслуживания: 1:25
